In [1]:
import pandas as pd
import string
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import wordnet
import nltk

import spacy
nlp = spacy.load('en')

import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import numpy as np
from scipy.sparse.csr import csr_matrix

from sklearn.feature_extraction.text import TfidfTransformer

# ЗАДАЕМ СЛОВАРЬ ДЛЯ РАЗВОРАЧИВАНИЯ СОКРАЩЕННЫХ ФРАЗ 

In [2]:
contr = pd.read_csv("./materials/contractions.csv")
contr.head()

,short,long
0,ain t,are not
1,aren t,are not
2,can t,cannot
3,can t ve,cannot have
4,'cause,because


In [3]:
contractions = {}
for key, val in zip(contr['short'],contr['long']):
    contractions[key] = val

# ЗАДАЕМ БАЗОВЫЙ СЛОВАРЬ

In [4]:
basic_vocabulary = []
with open("./materials/A1_vocab_processed.txt", "r") as voc:
    for word in voc.readlines():
        basic_vocabulary.append(word[:-1])
#basic_vocabulary = set(basic_vocabulary)
#basic_vocabulary

adjectives = []
with open("./materials/common_adj.txt", "r") as common_adj:
    for word in common_adj.readlines():
        adjectives.append(word[:-1])
        
common_uncountable = []
with open("./materials/common_unountable_manually_filtered.txt", "r") as common_unctbl:
    for word in common_unctbl.readlines():
        common_uncountable.append(word[:-1])

countries = []
with open("./materials/countries.txt", "r") as cntr:
    for word in cntr.readlines():
        countries.append(word[:-1])

names = []
with open("./materials/names.txt", "r") as names_file:
    for word in names_file.readlines():
        names.append(word[:-1])
        
print(len(basic_vocabulary), len(adjectives), len(common_uncountable), len(countries), len(countries), len(names))
final_basic_vocabulary = basic_vocabulary
final_basic_vocabulary.extend(adjectives)
final_basic_vocabulary.extend(common_uncountable)
final_basic_vocabulary.extend(countries)
final_basic_vocabulary.extend(names)
len(final_basic_vocabulary)

660 50 81 196 196 3309


4296

# ЗАДАЕМ СЛОВАРЬ ФРАЗОВЫХ ГЛАГОЛОВ

In [38]:
phrasal_list = []
with open("phrasal_verbs.txt", "r") as pv_doc:
    for pv in pv_doc:
        phrasal_list.append(pv[:-1])

# СЧИТЫВАЕМ ТЕКСТЫ

In [5]:
texts = pd.read_excel("./materials/texts.xlsx")
texts.head()

,id,Title,Text
0,1,Jimmy’s Breakfast,This is my friend Jimmy. He is from India. Jim...
1,2,A Girl from Green Valley,Green Valley was a small village near the Icy ...
2,3,Alice in Wonderland,Alice was beginning to get very tired of sitti...
3,4,Volkswagen,Volkswagen's emissions cheating will cost the ...
4,5,My Friends,My name is Sonya. I am 20 and I come from Moro...


# РАЗБИВАЕМ НА ПРЕДЛОЖЕНИЯ, ЧИСТИМ ПУНКТУАЦИЮ, ПРИВОДИМ К НИЖНЕМУ РЕГИСТРУ

In [6]:
puncuation_primary = string.punctuation + "’" +"“" + "”" 
puncuation_primary = puncuation_primary.replace(".","")

In [7]:
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
test_line = "I can't  visit U.S. and I couldn't   to go by your daddy's car. I want to say hello to her brother."#’'
lemmatizer = WordNetLemmatizer()
def punct_setnence_lower (line, sent_detector, debug = False, deep_debug = False):
    initial_sentences_list = sent_detector.tokenize(line.strip())
    final_sentences_list = []
    
    for sentence in initial_sentences_list:
        if(debug): print("sentence:", sentence)
        cleaned_line = ''
        #clean by word
        for word in sentence.split():
            if(debug): print(word)
            clean_word =''
            for char in word:
                if(deep_debug): print("char before cleaning:", char)
                if char not in puncuation_primary:
                    clean_word += char.lower()
                    if(deep_debug): print("char after cleaning:", char.lower())
                else:
                    clean_word += ' '
                    if(deep_debug): print("char has been deleted")
            clean_word = clean_word.lstrip()
            if(debug):print("non punctuation and lower:",clean_word)       

            if (clean_word in contractions):
                if(debug):print("word before contractions parsing:",clean_word)
                clean_word = ''.join(contractions[clean_word])
                if(debug):print("word after contractions parsing:",clean_word)
            if(debug): print("finally cleaned word/s:",clean_word)    
            cleaned_line += clean_word + ' '
        #handle abbreviations delete dots only in the end    
        for char_ind in range(len(cleaned_line) - 3, len(cleaned_line)):
            max_dot_index = len(cleaned_line)
            if cleaned_line[char_ind] == ".":
                if(char_ind < max_dot_index): max_dot_index = char_ind
                break
        cleaned_line = cleaned_line[:max_dot_index]    
        cleaned_line = re.sub(' +', ' ', cleaned_line)
        final_sentences_list.append(cleaned_line)
    return final_sentences_list
            
clean_line = punct_setnence_lower(test_line, sent_detector,debug = False)
clean_line

['i cannot visit u.s. and i could not to go by your daddy s car',
 'i want to say hello to her brother']

# ЗАДАЕМ ФУНКЦИЮ ДЛЯ ПОЛУЧЕНИЯ ГРАММАТИЧЕСКОЙ КАРТЫ ТЕКСТА

In [93]:
example_text = texts["Text"][0]
example_text_processed = punct_setnence_lower(example_text,sent_detector)
example_text_processed

['this is my friend jimmy',
 'he is from india',
 'jimmy is a vegetarian',
 'the best breakfast for him is a glass of orange juice two apples and three bananas',
 'jimmy likes watermelons very much',
 'a watermelon is green outside and red inside',
 'it is hard and soft inside',
 'it is juicy and sweet',
 'what a lovely fruit ',
 'a month ago sally received a letter',
 'it was an invitation for a wedding',
 'her sister victoria is getting married',
 'last year sally saw such a ceremony',
 'it took place on saturday',
 'there were a lot of guests people wore nice clothes met a husband and a wife near the church threw confetti ate a very tasty cake made videos and took photos',
 'in her letter victoria asked sally to be her bridesmaid',
 'it means that on the day of wedding sally must help victoria when she puts her wedding dress on and brushes her hair',
 'sally must also accompany victoria on her way to the church and carry a traditional bunch of flowers',
 'sally agreed with pleasure 

In [19]:
def get_sent_gramm_features_map(text):
    text_grammar_map = []
    for sentence in text:
        split_sent_list = sentence.split()
        grammar_map = [None] * len(split_sent_list)  
        parsed_sentence =  nlp(sentence)   
        assert (len(parsed_sentence) == len(grammar_map))
        for gramm_ind in range(len(split_sent_list)):
            #print(parsed_sentence[gramm_ind])
            if(parsed_sentence[gramm_ind].lemma_[0] == "-"):
                parsed_sentence[gramm_ind].lemma_ = parsed_sentence[gramm_ind].text
            grammar_map[gramm_ind] = (parsed_sentence[gramm_ind].lemma_, nlp.vocab.morphology.tag_map[parsed_sentence[gramm_ind].tag_])
        text_grammar_map.append(grammar_map)
    return text_grammar_map

grammar_map_example = get_sent_gramm_features_map(example_text_processed)
grammar_map_example[:2]

[[('this', {74: 89}),
  ('be',
   {74: 99,
    'Number': 'sing',
    'Person': 3,
    'Tense': 'pres',
    'VerbForm': 'fin'}),
  ('my', {74: 83, 'Poss': 'yes', 'PronType': 'prs'}),
  ('friend', {74: 91, 'Number': 'sing'}),
  ('jimmy', {74: 91, 'Number': 'sing'})],
 [('he', {74: 94, 'PronType': 'prs'}),
  ('be',
   {74: 99,
    'Number': 'sing',
    'Person': 3,
    'Tense': 'pres',
    'VerbForm': 'fin'}),
  ('from', {74: 84}),
  ('india', {74: 91, 'Number': 'sing'})]]

# ЛЕММАТИЗИРУЕМ ТЕКСТ

In [22]:
def get_lemm_text(text_gramm_map):
    lemm_text = []
    for sentence in text_gramm_map:
        sentence_lemm = ''
        for word in sentence:
            sentence_lemm += word[0] + ' '
        sentence_lemm = sentence_lemm[:-1]
        lemm_text.append(sentence_lemm)
        
    return lemm_text

ex_text_lemm = get_lemm_text(grammar_map_example)
ex_text_lemm[:3]

['this be my friend jimmy', 'he be from india', 'jimmy be a vegetarian']

# ВЫЧИСЛЯЕМ ТФИДФ

In [142]:
def get_tf_df_dataframe(lemm_text_list):
    vect = TfidfVectorizer(stop_words = 'english')
    tfidf_matrix = vect.fit_transform(lemm_text_list)
    df = pd.DataFrame(tfidf_matrix.toarray(), columns = vect.get_feature_names())
    print(df.head())
    
    return df
tf_df_dataframe = get_tf_df_dataframe (example_text_processed)

   accompany  adds  ago  agree  agreed    apples  asked  ate   bananas  \
0        0.0   0.0  0.0    0.0     0.0  0.000000    0.0  0.0  0.000000   
1        0.0   0.0  0.0    0.0     0.0  0.000000    0.0  0.0  0.000000   
2        0.0   0.0  0.0    0.0     0.0  0.000000    0.0  0.0  0.000000   
3        0.0   0.0  0.0    0.0     0.0  0.377964    0.0  0.0  0.377964   
4        0.0   0.0  0.0    0.0     0.0  0.000000    0.0  0.0  0.000000   

   beautiful  ...   wants  watermelon  watermelons  way  wedding  white  wife  \
0        0.0  ...     0.0         0.0     0.000000  0.0      0.0    0.0   0.0   
1        0.0  ...     0.0         0.0     0.000000  0.0      0.0    0.0   0.0   
2        0.0  ...     0.0         0.0     0.000000  0.0      0.0    0.0   0.0   
3        0.0  ...     0.0         0.0     0.000000  0.0      0.0    0.0   0.0   
4        0.0  ...     0.0         0.0     0.612137  0.0      0.0    0.0   0.0   

   wore  writes  year  
0   0.0     0.0   0.0  
1   0.0     0.0   0.

In [144]:
tf_df_dataframe.to_csv("./ex_text_tfidf.csv", sep = '\t')

# ЗАДАЕМ СПИСОК ВЕСОВ СЛОВ ДЛЯ ПОСЛЕДУЮЩЕГО ЗАПОЛНЕНИЯ ФИЧАМИ

In [122]:
def get_weights_empty_list(gramm_map_text):
    weights_list = []
    for sentence in gramm_map_text:
        #print(sentence)
        sentence_weights = []
        for element in sentence:
            weight = {"word" : element[0], "weight": 0}
            sentence_weights.append(weight)
        weights_list.append(sentence_weights)
    return weights_list
test_weights = get_weights_empty_list(grammar_map_example)
test_weights[:3]

[[{'weight': 0, 'word': 'this'},
  {'weight': 0, 'word': 'be'},
  {'weight': 0, 'word': 'my'},
  {'weight': 0, 'word': 'friend'},
  {'weight': 0, 'word': 'jimmy'}],
 [{'weight': 0, 'word': 'he'},
  {'weight': 0, 'word': 'be'},
  {'weight': 0, 'word': 'from'},
  {'weight': 0, 'word': 'india'}],
 [{'weight': 0, 'word': 'jimmy'},
  {'weight': 0, 'word': 'be'},
  {'weight': 0, 'word': 'a'},
  {'weight': 0, 'word': 'vegetarian'}]]

# ЗАДАЕМ ФУНКЦИЮ ДЛЯ ВЫЯВЛЕНИЯ СОЖНОЙ ГРАММАТИКИ

In [123]:
def get_difficult_grammar(text_grammar_map, weights_list, debug = False):
    for sentence_grammar_map, sentence_weights in zip(text_grammar_map,weights_list):
        for el_ind in range(len(sentence_grammar_map)):
            #print(sentence_grammar_map[el_ind])
            
            if('Aspect' in sentence_grammar_map[el_ind][1]):
                #present perfect
                if ( sentence_grammar_map[el_ind][1]['Aspect'] == 'perf' ):
                    if(sentence_grammar_map[el_ind - 2][0] == 'have' or sentence_grammar_map[el_ind - 1][0] == 'have'):
                        if(debug): print("PRESENT PERFECT")
                        if(debug): print(sentence_grammar_map[el_ind - 1])
                        if(debug): print(sentence_grammar_map[el_ind])

                        sentence_weights[el_ind]["diff_grammar"] = "pr_perf"
                        sentence_weights[el_ind - 1]["diff_grammar"] = "pr_perf"
                
                
                elif(sentence_grammar_map[el_ind][1]['Aspect'] == 'prog'):
                    #future continious check "will + be + v-ing"
                    if (sentence_grammar_map[el_ind - 1][0] == 'be' and sentence_grammar_map[el_ind - 2][0] == 'will'):
                        if(debug): print("FUTURE CONTINIOUS")                       
                        if(debug): print("prev word is", sentence_grammar_map[el_ind - 2])
                        if(debug): print("prev word is", sentence_grammar_map[el_ind - 1])
                        if(debug): print(sentence_grammar_map[el_ind])
                        sentence_weights[el_ind]["diff_grammar"] = "fut_cont"
                        sentence_weights[el_ind - 1]["diff_grammar"] = "fut_cont"
                        sentence_weights[el_ind - 2]["diff_grammar"] = "fut_cont"
                   #past continious was/were + v-ing"     
                    elif (sentence_grammar_map[el_ind - 1][0] == 'be' and sentence_grammar_map[el_ind -1][1]['Tense'] == 'past'):
                        if(debug): print("PAST CONTINIOUS")
                        if(debug): print(sentence_grammar_map[el_ind])
                        if(debug): print("prev word is", sentence_grammar_map[el_ind - 1])
                        sentence_weights[el_ind]["diff_grammar"] = "past_cont"
                        sentence_weights[el_ind - 1]["diff_grammar"] = "past_cont"
                        
                
    return weights_list

test_weights = get_difficult_grammar(grammar_map_example,test_weights, debug = True)
test_weights[:3]

[[{'weight': 0, 'word': 'this'},
  {'weight': 0, 'word': 'be'},
  {'weight': 0, 'word': 'my'},
  {'weight': 0, 'word': 'friend'},
  {'weight': 0, 'word': 'jimmy'}],
 [{'weight': 0, 'word': 'he'},
  {'weight': 0, 'word': 'be'},
  {'weight': 0, 'word': 'from'},
  {'weight': 0, 'word': 'india'}],
 [{'weight': 0, 'word': 'jimmy'},
  {'weight': 0, 'word': 'be'},
  {'weight': 0, 'word': 'a'},
  {'weight': 0, 'word': 'vegetarian'}]]

# ЗАДАЕМ ФНУКЦИЮ ДЛЯ ПОИСКА ФРАЗОВЫХ ГЛАГОЛОВ

In [124]:
def get_phrasal_verbs(text_grammar_map, weights_list, debug = False):
    for sentence_grammar_map, sentence_weights in zip(text_grammar_map,weights_list):
        for el_ind in range(1, len(sentence_grammar_map)):  
            for searh_ind in range(1, 3):
                try_phrase = sentence_grammar_map[el_ind - searh_ind][0] + ' ' + sentence_grammar_map[el_ind][0]
                #print(try_phrase)
                if(try_phrase in phrasal_list):
                    if(debug): print("Phrasal Verb found:", try_phrase)
                    sentence_weights[el_ind]["phrasal_verb"] = "yes"
                    sentence_weights[el_ind - searh_ind]["phrasal_verb"] = "yes"
    return weights_list
        #print("\n")
test_weights = get_phrasal_verbs(grammar_map_example,test_weights,debug = True)
test_weights[-2:]

Phrasal Verb found: agree with
Phrasal Verb found: add up
Phrasal Verb found: give up


[[{'weight': 0, 'word': 'she'},
  {'weight': 0, 'word': 'be'},
  {'weight': 0, 'word': 'dream'},
  {'weight': 0, 'word': 'about'},
  {'weight': 0, 'word': 'a'},
  {'weight': 0, 'word': 'slice'},
  {'weight': 0, 'word': 'of'},
  {'weight': 0, 'word': 'wedding'},
  {'weight': 0, 'word': 'cake'},
  {'weight': 0, 'word': 'and'},
  {'weight': 0, 'word': 'do'},
  {'weight': 0, 'word': 'not'},
  {'weight': 0, 'word': 'want'},
  {'weight': 0, 'word': 'to'},
  {'phrasal_verb': 'yes', 'weight': 0, 'word': 'give'},
  {'phrasal_verb': 'yes', 'weight': 0, 'word': 'up'}],
 [{'weight': 0, 'word': 'dieting'},
  {'weight': 0, 'word': 'be'},
  {'weight': 0, 'word': 'not'},
  {'weight': 0, 'word': 'easy'}]]

# ПРИСВАИВАЕМ ВЕСА ТФИДФ

In [145]:
tf_df_dataframe.head()

,accompany,adds,ago,agree,agreed,apples,asked,ate,bananas,beautiful,...,wants,watermelon,watermelons,way,wedding,white,wife,wore,writes,year
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.377964,0.0,0.0,0.377964,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.612137,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [147]:
tf_idf_dict = tf_df_dataframe.to_dict()

In [148]:
def assign_tf_idf(text_grammar_map, weights_list, tf_idf_dict):
    assert (len(text_grammar_map) == len(weights_list))
    for sentence_ind in range(len(text_grammar_map)):
        for el_ind in range(len(text_grammar_map[sentence_ind])):
            if (weights_list[sentence_ind][el_ind]["word"] in tf_idf_dict):
                weights_list[sentence_ind][el_ind]["weight"] = tf_idf_dict[weights_list[sentence_ind][el_ind]["word"]][sentence_ind]
            else:
                weights_list[sentence_ind][el_ind]["weight"] = 0.01
                
    return weights_list
test_weights = assign_tf_idf(grammar_map_example, test_weights, tf_idf_dict)
test_weights[:3]        

[[{'weight': 0.01, 'word': 'this'},
  {'weight': 0.01, 'word': 'be'},
  {'weight': 0.01, 'word': 'my'},
  {'weight': 0.774119582428778, 'word': 'friend'},
  {'weight': 0.6330393922184421, 'word': 'jimmy'}],
 [{'weight': 0.01, 'word': 'he'},
  {'weight': 0.01, 'word': 'be'},
  {'weight': 0.01, 'word': 'from'},
  {'weight': 1.0, 'word': 'india'}],
 [{'weight': 0.6330393922184421, 'word': 'jimmy'},
  {'weight': 0.01, 'word': 'be'},
  {'weight': 0.01, 'word': 'a'},
  {'weight': 0.774119582428778, 'word': 'vegetarian'}]]

# ДЕЛИМ НА СЛОЖНЫЕ И НЕ СЛОЖНЫЕ В СООТВЕТСВИИ С ПОЛУЧЕННЫМИ ФИЧАМИ

In [151]:
def split_into_groups(text_weights):
    difficult_vocabulary = []
    easy_vocabulary = []
    for sentence_weights in text_weights:
        for word_weight in sentence_weights:
            #print(word_weight)
            if('diff_grammar' in word_weight or 'phrasal_verb' in word_weight or word_weight['word'] not in basic_vocabulary):
                difficult_vocabulary.append(word_weight)
            else:
                easy_vocabulary.append(word_weight)
    return easy_vocabulary, difficult_vocabulary
                
easy, difficult = split_into_groups(test_weights)

In [152]:
len(easy), len(difficult)

(252, 62)

In [153]:
difficult

[{'weight': 0.774119582428778, 'word': 'vegetarian'},
 {'weight': 0.0, 'word': 'watermelon'},
 {'weight': 0.4664399378418825, 'word': 'watermelon'},
 {'weight': 0.5978117297300675, 'word': 'soft'},
 {'weight': 0.7457391260354046, 'word': 'juicy'},
 {'weight': 0.7457391260354046, 'word': 'lovely'},
 {'weight': 0.01, 'word': 'receive'},
 {'weight': 0.774119582428778, 'word': 'invitation'},
 {'weight': 0.6330393922184421, 'word': 'wedding'},
 {'weight': 0.01, 'word': 'marry'},
 {'weight': 0.01, 'word': 'such'},
 {'weight': 0.49926416816836866, 'word': 'ceremony'},
 {'weight': 0.01, 'word': 'guest'},
 {'weight': 0.01, 'word': 'clothe'},
 {'weight': 0.20945709765470802, 'word': 'church'},
 {'weight': 0.01, 'word': 'throw'},
 {'weight': 0.23445125960680535, 'word': 'confetti'},
 {'weight': 0.23445125960680535, 'word': 'tasty'},
 {'weight': 0.01, 'word': 'video'},
 {'weight': 0.5144649816247601, 'word': 'bridesmaid'},
 {'weight': 0.01, 'word': 'mean'},
 {'weight': 0.5058593576427194, 'word': 

In [59]:
def calculate_level(easy_words_weights, difficults_words_weights, debug = False):
    tf_idf_easy = 0
    for easy_w_w in easy_words_weights:
        tf_idf_easy += easy_w_w['weight']
        
    tf_idf_diff = 0
    for diff_w_w in difficults_words_weights:
        tf_idf_diff += diff_w_w['weight']
        
    if(debug):print(len(easy_words_weights), len(difficults_words_weights))
    if(debug):print(tf_idf_easy, tf_idf_diff)
    overall_weights = tf_idf_easy + tf_idf_diff
    if(debug):print(round(tf_idf_easy/overall_weights, 2), round(tf_idf_diff/overall_weights, 2))
    if(round(tf_idf_diff/overall_weights, 2) >= 0.3):
        print("text is very diffcult, diff realtive weight is more then 0.3")
    if(round(tf_idf_diff/overall_weights, 2) > 0.2 and round(tf_idf_diff/overall_weights, 2) < 0.3):
        print("text is moderate")
    if(round(tf_idf_diff/overall_weights, 2) <= 0.2):
        print("text is quite easy")
calculate_level(easy, difficult)

text is very diffcult, diff realtive weight is more then 0.3


In [87]:
def calculate_text_level(raw_text, print_debug_message = False, show_calucated_weights = False):
    processed_text = punct_setnence_lower(raw_text,sent_detector)
    
    grammar_map = get_sent_gramm_features_map(processed_text)
    
    text_lemm = get_lemm_text(grammar_map)
    
    tfidf_dict = get_tf_idf_dict(text_lemm)
    print(tfidf_dict)
    
    weights = get_weights_empty_list(grammar_map)
    
    weights = get_difficult_grammar(grammar_map,weights, debug = print_debug_message)
    weights = get_phrasal_verbs(grammar_map,weights, debug = print_debug_message)
    weights = assign_tf_idf(grammar_map,weights, tfidf_dict)
    if(show_calucated_weights): 
        print("WEIGHTS HAVE BEEN CALCULATED AS FOLLOWS")
        for sent_weights in weights:
            for word_weight in sent_weights:
                print(word_weight)
            print("\n")
    easy, difficult = split_into_groups(weights)
    
    calculate_level(easy, difficult,  debug = print_debug_message)

In [88]:
calculate_text_level(texts['Text'][2],print_debug_message = True, show_calucated_weights = True)

{'white': 1.0, 'way': 1.0, 'pleasure': 1.0, 'much': 1.0, 'make': 1.0, 'day': 1.0, 'could': 1.0, 'sister': 0.778, 'get': 0.628}
PAST CONTINIOUS
('begin', {74: 99, 'VerbForm': 'part', 'Tense': 'pres', 'Aspect': 'prog'})
prev word is ('be', {74: 99, 'VerbForm': 'fin', 'Tense': 'past'})
PRESENT PERFECT
('have', {74: 99, 'VerbForm': 'fin', 'Tense': 'past'})
('peep', {74: 99, 'VerbForm': 'part', 'Tense': 'past', 'Aspect': 'perf'})
PAST CONTINIOUS
('read', {74: 99, 'VerbForm': 'part', 'Tense': 'pres', 'Aspect': 'prog'})
prev word is ('be', {74: 99, 'VerbForm': 'fin', 'Tense': 'past'})
PAST CONTINIOUS
('consider', {74: 99, 'VerbForm': 'part', 'Tense': 'pres', 'Aspect': 'prog'})
prev word is ('be', {74: 99, 'VerbForm': 'fin', 'Tense': 'past'})
WEIGHTS HAVE BEEN CALCULATED AS FOLLOWS
{'word': 'alice', 'weight': 0.1}
{'word': 'be', 'weight': 0.1, 'diff_grammar': 'past_cont'}
{'word': 'begin', 'weight': 0.1, 'diff_grammar': 'past_cont'}
{'word': 'to', 'weight': 0.1}
{'word': 'get', 'weight': 0.628